## Profiler: why the function is single-threaded?

In [ ]:
from Hubbard.equalizer import *
import numpy as np
import matplotlib.pyplot as plt

N = 20
R0 = np.array([3, 3, 7.2])
G = HubbardParamEqualizer(N,
                          R0=R0,
                          lattice=np.array([4], dtype=int),
                          band=1,
                          dim=2,
                          avg=1 / 2,
                          sparse=True,
                          equalize=False,
                          symmetry=True,
                          verbosity=0)
%prun __, __, __, info = G.equalzie('uvt', random=True, callback=True)


Initial trap depths: [0.94285541 0.98085785]
Initial waists:
[[1.08291754 1.        ]
 [0.98205364 1.        ]]
Initial trap centers:
[[-1.58238511  0.        ]
 [-0.4082596   0.        ]]
i=50	c=[0.09990731 0.9167044  0.07502301]	c_i=1.0916347190668363	c_i//2-c_i=-0.49423491499167793
i=100	c=[0.00121951 0.02686512 0.0065927 ]	c_i=0.03467733374624908	c_i//2-c_i=0.3542075872553148
i=150	c=[0.00163742 0.0015902  0.00064233]	c_i=0.0038699418280335653	c_i//2-c_i=0.04911866546302507
i=200	c=[9.74662268e-06 3.73898051e-05 1.17859688e-04]	c_i=0.00016499611579848136	c_i//2-c_i=0.03451234249638431
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.389232003670117e-06
            Iterations: 29
            Function evaluations: 249
            Gradient evaluations: 29
 

         6482823 function calls (6396441 primitive calls) in 14.913 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   112940    7.377    0.000   11.540    0.000 arpack.py:533(iterate)
   339820    2.188    0.000    2.188    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
406919/377559    1.237    0.000    4.274    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   112940    1.099    0.000    3.893    0.000 core.py:390(H_op)
    12000    0.492    0.000    0.531    0.000 function_base.py:3543(sinc)
      500    0.217    0.000    2.573    0.005 core.py:576(wannier_func)
   366781    0.175    0.000    0.175    0.000 {method 'reshape' of 'numpy.ndarray' objects}
   112940    0.147    0.000    4.184    0.000 _interface.py:201(matvec)
     6000    0.121    0.000    0.670    0.000 core.py:303(delta)
      500    0.107    0.000    0.108    0.000 arpack.py:573(extract)
  1699100    0.106  

In [ ]:
from DVR.core import *
from numba import guvectorize, float64, int64

p = np.array([1,  1,  1])
x = [np.linspace(0, 1, 10), np.linspace(0, 1, 10), np.zeros(10)]
xn = [np.linspace(0, 1, 10), np.linspace(0, 1, 10), np.zeros(10)]


def f1(p, x, xn):
    V = map(delta, p, x, xn)
    return contract('ij,kl,mn', *V)


def f2(p, x, xn):
    vd = np.vectorize(delta, signature='(),(m),(n)->(m,n)')
    V = vd(p, x, xn)
    return contract('ij,kl,mn', *V)


@guvectorize([(int64, float64[:], float64[:], float64[:, :])], '(),(m),(n)->(m,n)', target='parallel')
def numbadelta(p: int, x: np.ndarray, xn: np.ndarray, Wx):
    # Symmetrized sinc DVR basis funciton, x in unit of dx
    x = x.copy()
    xn = xn.copy()
    Wx = np.sinc(x.reshape(-1, 1) - xn.reshape(1, -1))
    if p != 0:
        Wx += p * np.sinc(x.reshape(-1, 1) + xn.reshape(1, -1))
        Wx /= np.sqrt(2)
        if p == 1:
            Wx[:, 0] /= np.sqrt(2)


def f3(p, x, xn):
    V = numbadelta(p, x, xn)
    return contract('ij,kl,mn', *V)


%timeit V = f1(p, x[:2] + [np.array([0])], xn[:2] + [np.array([0])])
%timeit V = f2(p, x[:2] + [np.array([0.])], xn[:2] + [np.array([0.])])
# %timeit V = f2(p, x, xn)
%timeit V = f3(p, x, xn)


117 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


/Users/nottforestfc/mambaforge/lib/python3.9/site-packages/numpy/lib/function_base.py:2410: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args = tuple(asanyarray(arg) for arg in args)


TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable sin method